In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
from os.path import isfile

In [10]:
time_list = '05,11,17,23'.split(',')

#  2015-12-29
# day_list = [d.strftime('%Y%m%d') for d in pd.date_range(start='2015-12-29', end='2019-12-10')]
day_list = [d.strftime('%Y%m%d') for d in pd.date_range(start='2016-01-01', end='2016-08-07')]
day_list = list(reversed(day_list))
day_list

['20160807',
 '20160806',
 '20160805',
 '20160804',
 '20160803',
 '20160802',
 '20160801',
 '20160731',
 '20160730',
 '20160729',
 '20160728',
 '20160727',
 '20160726',
 '20160725',
 '20160724',
 '20160723',
 '20160722',
 '20160721',
 '20160720',
 '20160719',
 '20160718',
 '20160717',
 '20160716',
 '20160715',
 '20160714',
 '20160713',
 '20160712',
 '20160711',
 '20160710',
 '20160709',
 '20160708',
 '20160707',
 '20160706',
 '20160705',
 '20160704',
 '20160703',
 '20160702',
 '20160701',
 '20160630',
 '20160629',
 '20160628',
 '20160627',
 '20160626',
 '20160625',
 '20160624',
 '20160623',
 '20160622',
 '20160621',
 '20160620',
 '20160619',
 '20160618',
 '20160617',
 '20160616',
 '20160615',
 '20160614',
 '20160613',
 '20160612',
 '20160611',
 '20160610',
 '20160609',
 '20160608',
 '20160607',
 '20160606',
 '20160605',
 '20160604',
 '20160603',
 '20160602',
 '20160601',
 '20160531',
 '20160530',
 '20160529',
 '20160528',
 '20160527',
 '20160526',
 '20160525',
 '20160524',
 '20160523',

In [12]:
def parse_page(html, filename):
    soup = BeautifulSoup(html, 'lxml')

    result = []
    for tag in soup.select('dl.forecast'):    
        texts = [t.get_text().strip() for t in tag.select('div.txtbox')]
        
        if len(texts) == 0:
            continue
        
        item = {
            'day': day,
            'time': t,
            'when': [t.get_text().strip() for t in tag.select('dt')][0],
            'text': texts[0]
        }
        
        if len(texts) > 1:
            item['reason'] = texts[1]

        result.append(item)
        
    if filename is None:
        return result
    
    with open(filename, 'w') as fp:
        fp.write(str(soup))

    return result

def get_html(ymd):
    url = 'http://www.airkorea.or.kr/web/dustForecast?pMENU_NO=113'
    
    post_data = {
        'loading': 'yes',
        'leftShow': 'realTime',
        'ymd': day + t
    }
    
    proxies = {
        'http': 'socks5://localhost:9150',
        'https': 'socks5://localhost:9150',
    }    

    resp = requests.post(url, data=post_data, proxies=proxies)
        
    return resp.text

In [13]:
rows = []

In [14]:
for day in tqdm(day_list):
    for t in tqdm(time_list, desc=day):
        filename = 'data/airkorea/{day}-{time}.html'.format(day=day, time=t)
        if isfile(filename) is True:
            with open(filename, 'r') as fp:
                html = ''.join(fp.readlines())
            
            filename = None
        else:
            try:
                html = get_html(ymd=day + t)
                sleep(4)
            except Exception as e:
                print(e)
            
        rows += parse_page(html=html, filename=filename)







  0%|          | 0/220 [00:00<?, ?it/s]






20160807:   0%|          | 0/4 [00:00<?, ?it/s]






20160807: 100%|██████████| 4/4 [00:00<00:00, 36.43it/s]






  0%|          | 1/220 [00:00<00:24,  8.91it/s]






20160806:   0%|          | 0/4 [00:00<?, ?it/s]






20160806:  75%|███████▌  | 3/4 [00:00<00:00, 26.99it/s]






20160806: 100%|██████████| 4/4 [00:04<00:00,  1.10s/it]






  1%|          | 2/220 [00:04<05:04,  1.40s/it]






20160805:   0%|          | 0/4 [00:00<?, ?it/s]

('Connection broken: IncompleteRead(2914 bytes read, 5144 more expected)', IncompleteRead(2914 bytes read, 5144 more expected))









20160805:  25%|██▌       | 1/4 [00:02<00:06,  2.33s/it]

('Connection broken: IncompleteRead(2914 bytes read, 5144 more expected)', IncompleteRead(2914 bytes read, 5144 more expected))









20160805:  50%|█████     | 2/4 [00:04<00:04,  2.30s/it]

('Connection broken: IncompleteRead(2914 bytes read, 5144 more expected)', IncompleteRead(2914 bytes read, 5144 more expected))


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(rows)

In [ ]:
df.head()

In [ ]:
df['day'].min(), df['day'].max()

In [ ]:
filename = 'data/airkorea-{}~{}.xlsx'.format(df['day'].min(), df['day'].max())
df.to_excel(filename, index=False)